# ECE 209AS CA5

Zan Xie 205364923

In [ ]:
# library
!pip install gspread
import gspread
from google.colab import auth
from google.auth import default
auth.authenticate_user()

import re
import hashlib
import datetime

In [132]:
# compute SHA256 hash
def SHA256(input_string):
  return hashlib.sha256(input_string.encode()).hexdigest()

# proof of work function
def proof_of_work(data, difficulty='0000'):
  nonce = 0
  while True:
    nonce_str = hex(nonce)[2:]
    hash_result = SHA256(data + nonce_str)
    if hash_result.startswith(difficulty):
      return nonce_str, hash_result
    nonce += 1

# convert column index to letter notation
def index_to_letter(index):
  letters = ''
  while index > 0:
    index, remainder = divmod(index - 1, 26)
    letters = chr(65 + remainder) + letters
  return letters


In [134]:
# load in worksheet
file_name = 'myBlockchainPython'
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open(file_name).sheet1

all_values = worksheet.get_all_values()
last_col = len(all_values[0])

# the most recent block hash
# hardcode the most recent block's data and get its hash instead of retriving it using get_value()
# detail explaination in project report
pre_block_index = int(worksheet.cell(1, last_col - 2).value.split('#')[1])
if (pre_block_index == 0):
  data = 'Block #0TransactionsFromToAmount-Miner #112.5MeadowPJ15.13PJNader75.52MeadowEllie31.51VictimHacker99MetadataPrevious Hash2e6e504eaf47df8e4c7c9d7109073a2e        Timestamp45051.7404976852Nonce24e'
  pre_hash = SHA256(data)
else:
  pre_hash = worksheet.cell(14, last_col - 1).value

print('pre_block_idx:', pre_block_index)
print('pre_hash:',pre_hash)


pre_block_idx: 1
pre_hash: 0000ea4b0bcf46f11fc3cd8cfc96a570b9b4a6b710c061f67b4bbc673b661d3c


In [135]:
# generate a transcation block
start_col = index_to_letter(last_col + 1)
end_col = index_to_letter(last_col + 3)
block_range = f"{start_col}1:{end_col}14"
block_data = [['' for _ in range(3)] for _ in range(14)]

# transcation record, minic real-transcation
pay = ['-', 'Stan']
receive = ['Miner #1', 'Bob']
amount = ['12.50', '9.90']

# datetime
block_index = pre_block_index + 1
now = datetime.datetime.now()
format_datetime = now.strftime("%m/%d/%Y %H:%M:%S")

# Set values in the block
block_data[0][0] = 'Block #' + str(block_index)
block_data[1][0] = 'Transactions'
block_data[2][0] = 'From'
block_data[2][1] = 'To'
block_data[2][2] = 'Amount'

# transcation content
trans_idx = 3
trans_limit = 7
for x,y,z in zip(pay, receive, amount):
  if (trans_idx <= trans_limit):
    block_data[trans_idx][0] = x
    block_data[trans_idx][1] = y
    block_data[trans_idx][2] = z
  trans_idx+=1

# metadata section
block_data[9][0] = 'Metadata'
block_data[10][0] = 'Previous Hash'
block_data[10][1] = pre_hash
block_data[11][0] = 'Timestamp'
block_data[11][1] = format_datetime
block_data[12][0] = 'Nonce'
block_data[13][0] = 'Hash'

# join block contents togeter and pass to hashing
allData = ''.join(''.join(row) for row in block_data)
print(allData)

nonce, hash_val = proof_of_work(allData)
print('nonce:',nonce)
print('hash:',hash_val)

# write nonce and hash into cell
block_data[12][1] = nonce
block_data[13][1] = hash_val

# Update the worksheet
worksheet.update(range_name=block_range, values=block_data)


Block #2TransactionsFromToAmount-Miner #112.50StanBob9.90MetadataPrevious Hash0000ea4b0bcf46f11fc3cd8cfc96a570b9b4a6b710c061f67b4bbc673b661d3cTimestamp03/20/2024 18:24:06NonceHash
nonce: 1dda
hash: 0000fd253f2d8a6c54a280a04eb40c5902234d66a697132e6c543af258e29907


{'spreadsheetId': '15aRaxgSMDKUKe0s4ND1V1UarUc7wQZYgvm27BEL0p9s',
 'updatedRange': "'Block Structure'!G1:I14",
 'updatedRows': 14,
 'updatedColumns': 3,
 'updatedCells': 42}